In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
mainURL = "https://www.programiz.com/swift-programming"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'class': 'right-bar'}).findAll(href=True)
for slug in pages[:-7]:
    urls.append('https://www.programiz.com' + slug.get('href'))

mainHeadings = []
mainContents = []
videoHeadings = []
videoURLs = []
subHeadingsH2 = []
subHeadingsH3 = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
snippets = []
outputs = []
tableData = []
listItems = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    mainHeading = soup.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    mainContent = soup.findAll('p', {"class": "editor-contents__short-description"})  # Main Content
    pageMainContents = []
    if mainContent is not None:
        for mainCon in mainContent:
            pageMainContents.append(mainCon.text)

    videoHeading = soup.findAll('div', {"class": "programiz-youtube-wrapper lite-youtube"})  # Video Heading
    pageVideoHeadings = []
    if videoHeading is not None:
        for vidHead in videoHeading:
            pageVideoHeadings.append(vidHead.find('h2').text)

    videoURL = soup.findAll('div', {"class": "programiz-youtube__container"})  # Video URL
    pageVideoURLs = []
    if videoURL is not None:
        for vid in videoURL:
            pageVideoURLs.append(vid.find('iframe').get('src'))

    subContent = soup.find('div', {"class": "content"})  # Sub Content
    if subContent is None:
        continue

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append(img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text
            if 5 <= len(slug) <= 65:
                pageFacts.append(slug)
            elif 66 <= len(slug) <= 126:
                pageDefinitions.append(slug)
            elif 127 < len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('pre')  # Codes, Snippets and Outputs
    pageCodes = []
    pageSnippets = []
    pageOutputs = []
    for slug in codeContent:
        if slug.code is not None:
            for cod in slug.code.text.split('\n'):
                if cod != '' and not cod.lstrip().rstrip().startswith('//'):
                    if 7 < len(cod) <= 30:
                        pageSnippets.append(cod)
                    elif 31 < len(cod):
                        pageCodes.append(cod)

        else:
            for out in slug.text.split('\n'):
                if out != '' and not out.lstrip().rstrip().startswith('//'):
                    pageOutputs.append(out)
                else:
                    continue

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text).replace('\n', '|||').replace('\t\t', '')

        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)

    mainHeadings.append(pageMainHeadings)
    mainContents.append(pageMainContents)
    videoHeadings.append(pageVideoHeadings)
    videoURLs.append(pageVideoURLs)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    snippets.append(pageSnippets)
    codes.append(pageCodes)
    outputs.append(pageOutputs)
    tableData.append(pageTableData)
    listItems.append(pageListItems)


In [4]:
data = {'main_headings': mainHeadings, 'main_contents': mainContents, 'video_headings': videoHeadings,
        'video_urls': videoURLs, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'image_urls': imageURLs, 'facts': facts, 'definitions': definitions, 'explanations': explanations,
        'snippets': snippets, 'codes': codes, 'outputs': outputs, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('Programiz_Swift.csv')

In [5]:
a=pd.read_csv("Programiz_Swift.csv")
a

,Unnamed: 0,main_headings,main_contents,video_headings,video_urls,sub_headings_h2,sub_headings_h3,image_urls,facts,definitions,explanations,snippets,codes,outputs,table_data,list_items
0,0,"[' Swift “Hello, World!” Program ']","['In this tutorial, you will learn to write a ...",[],[],"['Swift ""Hello, World!"" Program', 'How Swift ""...",[],[],"['Let\'s explore how to create ""Hello, World!""...",['Here are the different sections of the Hello...,['A Hello World is a simple program that print...,"['print(""Hello, World!"") ', 'print(""Hello, Wor...",[],"['Hello, World!']",[],[]
1,1,"['Swift Variables, Constants and Literals']","['In this tutorial, you will learn about varia...",[],[],"['1. Swift Variables', 'Declare Variables in S...",[],[],"['Here, num is a variable storing the value 10...","['In programming, a variable is a container (s...",['Literals are representations of fixed values...,"['var num = 10 ', 'var siteName:String', 'var ...","['print(siteName) // programiz.com', 'var 1a =...","['programiz.com', 'apple.com', ""main.swift:4:1...","['ype|||Example|||Remarks&&&ecimal|||5, 10, -6...",['siteName is a variable of type String. Meani...
2,2,['Swift Data Types'],"['In this tutorial, you will learn about data ...",[],[],"['Swift Data Types', 'Swift Character', 'Swift...",['Variants of Int type'],[],['There are six basic types of data types in S...,"['In computer programming, data types specify ...",['The character data type is used to represent...,"['var num: Int', 'var letter: Character = ""s""'...",['let latitude: Double = 27.7007697012432'],"['true', 'false']",['ata Types|||Example|||Description&&&haracter...,['Size: Depends on the platform type|||Range: ...
3,3,['Swift Characters and Strings'],"[""In this tutorial, you will learn about Swift...",[],[],"['Swift Character', 'Character Example', 'Swif...","['1. Compare Two Strings', '2. Join Two String...",[],"['Here, the letter variable can only store sin...",['Character is a data type that represents a s...,"['In the above example, we have created two ch...","['var letter: Character', 'var letter: Charact...","['var example = ""This is ""String"" class""', 'va...","['Swift', 'I love Swift.', 'false', 'true', 'H...",['uilt-in function|||Description&&&sEmpty|||de...,"['str1 and str2 are not equal. Hence, the resu..."
4,4,['Swift Basic Input and Output'],"['In this tutorial, we will learn simple ways ...",[],[],"['Swift Output', 'Example 1: Swift Print State...",[],[],"['Syntax of print()', 'Here,', 'Output', 'So w...","['In Swift, we can simply use the print() func...",['Note: separator and terminator are optional....,"['print(""Swift is powerful"")', 'print(""Good Mo...","['print(items: separator: terminator:)', 'prin...","['Good Morning!', ""It's rainy today"", ""Good Mo...",[],['items - values inside the double quotation||...
5,5,"['Swift Expressions, Statements and Code blocks']","['In this article, you will learn about Swift ...",[],[],"['Swift Expressions', 'Swift Statements', 'Swi...","['1. Simple Statements', '2. Conditional State...",[],"['Here, expressions are:', 'There are three ty...","['An expression is a combination of variables,...",['Simple statements are the most common types ...,"['var marks = 80', 'var result = (num1 == num2...",[],"['Hello, World!', 'Hello, World!', 'Hello, Wor...",[],['var marks = 80 - represents we are assigning...
6,6,['Swift Comments'],"['In this article, you will learn about Swift ...",[],[],"['Single Line Comment', 'Multiline Comment', '...",[],[],['There are two ways to add comments in Swift:...,"['In computer programming, comments are hints ...",['If we get an error while running the program...,"['var name = ""Cartman""', 'print(name)', '/* cr...","['var name = ""swift"" // name is a string']",[],[],['// - Single Line comments|||/*...*/ - Multil...
7,7,['Swift Optionals'],"['In this article, you will learn about option...",[],[],"['How to declare an Optional?', 'Optional Hand...",['Example 1: How to declare an optional in Swi